In [1]:
!pip install spacy
!pip install nltk

In [2]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 4.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import torch
import torchtext

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset,DataLoader
from torchtext.datasets import multi30k,Multi30k
from torch.nn.utils.rnn import pad_sequence # New

from torchdata.datapipes.iter import IterableWrapper,Mapper # New

import torch.nn as nn

import numpy as np
import pandas as pd
import random
import time # New
import matplotlib.pyplot as plt


def warn(*args,**kwargs):
    pass

import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


In [4]:
w_xh = torch.tensor(-10.0)
w_hh = torch.tensor(10.0)
b_h = torch.tensor(0.0)
x_t =1
h_prev = torch.tensor(-1)

In [5]:
X = [1,1,-1,-1,1,1]

Assuming that you start from the intial state $h = 0$,  with the above input vector $x$, the state vector $h$ should look like this:


In [6]:
H=[-1,-1,0,1,0,-1]

In [7]:
# Initialize an empty list to store the predicted state values
H_hat = []

# Loop through each data point in the input sequence X
t =1

for x in X:
    # assign the current data point to x_t
    print("t =",t)
    x_t = x

    # Print the value of the previous state (h at time t-1)
    print("h_t-1",h_prev.item())


    # Compute the current state (h at time t) using the RNN formaula with tanh activation
    h_t = torch.tanh(x_t*w_xh +h_prev * w_hh +b_h)


    # update the h_prev to the current state value for the next iter
    h_prev = h_t

    # print the current input value (x at time t)
    print(f"h_t:{h_t.item()}\n")

    ## Append the current state value to the H_hat list after converting it to the integer
    H_hat.append(int(h_t.item()))

    t+=1
    

t = 1
h_t-1 -1
h_t:-1.0

t = 2
h_t-1 -1.0
h_t:-1.0

t = 3
h_t-1 -1.0
h_t:0.0

t = 4
h_t-1 0.0
h_t:1.0

t = 5
h_t-1 1.0
h_t:0.0

t = 6
h_t-1 0.0
h_t:-1.0



In [8]:
H_hat,H

([-1, -1, 0, 1, 0, -1], [-1, -1, 0, 1, 0, -1])

## Sequence2Sequence

### Encoder

In [84]:
class Encoder(torch.nn.Module):
    def __init__(self,vocab_len,emb_dim,hid_dim,n_layers,dropout_prob):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(vocab_len, emb_dim)

        self.lstm = nn.LSTM(emb_dim,hid_dim,n_layers,dropout = dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)


    def forward(self,input_batch):

        # input_batch = [src_len, batch_size]

        embed = self.dropout(self.embedding(input_batch))

        embed = embed.to(device)

        # outputs = [src_len, batch_size, hid_dim*n directions]
        # hidden = [n_layers * n directions, batch_size, hid_dim]
        # cell = [n_layers * n directions, batch_size, hid dim]

        outputs, (hidden,cell) = self.lstm(embed)

        return hidden,cell

        

In [10]:
'''class Encoder(torch.nn.Module):

    def __init__(self,vocab_size,embed_dim,hid_dim,n_layers,dropout_prob):

        super().__init__()

        self.dropout = nn.Dropout(dropout_prob)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim,hid_dim,n_layers,dropout = dropout_prob)


    def forward(self,input_batch):
        # input batch size = [each_sentence_length,batch_size]
        embed = self.dropout(self.embedding(input_batch))

        embed = embed.to(device)

        outputs, (hidden,cell) = self.lstm(embed)

        return hidden,cell'''

'class Encoder(torch.nn.Module):\n\n    def __init__(self,vocab_size,embed_dim,hid_dim,n_layers,dropout_prob):\n\n        super().__init__()\n\n        self.dropout = nn.Dropout(dropout_prob)\n        self.embedding = nn.Embedding(vocab_size, embed_dim)\n        self.lstm = nn.LSTM(embed_dim,hid_dim,n_layers,dropout = dropout_prob)\n\n\n    def forward(self,input_batch):\n        # input batch size = [each_sentence_length,batch_size]\n        embed = self.dropout(self.embedding(input_batch))\n\n        embed = embed.to(device)\n\n        outputs, (hidden,cell) = self.lstm(embed)\n\n        return hidden,cell'

Now create an encoder instance to see how it works

In [85]:
vocab_len = 8
emb_dim = 10
hid_dim = 8
n_layers = 1
dropout_prob = 0.5
device = torch.device('cpu')

encoder_t = Encoder(vocab_len, emb_dim, hid_dim, n_layers, dropout_prob).to(device)

In [86]:
src_batch = torch.tensor([[0,3,4,2,1]]) # Input of single token
# you need to transpose the input tensor as the encoder LSTM is in sequence_first mode
src_batch = src_batch.t().to(device)

## Input sequence has two options: 1) batch first & 2) Sequence first
### In case of Sequence first input tensor should be transposed, otherwise not

print(f"Shape of input (src) tensor: {src_batch.shape}")

hidden_t, cell_t = encoder_t(src_batch)
print(f"Hidden tensor from encoder: {hidden_t}\nCell tensor from encodder: {cell_t}")

Shape of input (src) tensor: torch.Size([5, 1])
Hidden tensor from encoder: tensor([[[-0.5584,  0.1026, -0.0183,  0.0380, -0.0150, -0.0235,  0.0213,
           0.0561]]], grad_fn=<StackBackward0>)
Cell tensor from encodder: tensor([[[-0.8698,  0.3432, -0.0218,  0.0572, -0.0221, -0.1568,  0.0439,
           0.0870]]], grad_fn=<StackBackward0>)


The encoder takes the input sentences which consists of a sequence of words or tokens. The encoder LSTM process the entire input sequence and updates it hidden states at each time step. The hidden states of the LSTM network act as a form of memory and capture the contextual information of the input sequence. After processing the entire input sequence, the final hidden state of the encoder LSTM captures the summarized representation of the input sequences's context. This final hidden state is sometimes referred to as the `"Context vector"`

In [87]:
hidden_t.shape,cell_t.shape,src_batch.shape 

# Shape verification
# input_shape = [5,1] --> one sentence consists of 5 words--> batch_size =1, src_len = 5
# output_shape = [] --> src_len, batch_size, hid_dim*n_directions --> 5,1,8
# hidden_shape = [] --> n_layers*n_direction, batch_size , hid_dim --> 1,1,8
# cell_shape = [] --> 1,1,8

(torch.Size([1, 1, 8]), torch.Size([1, 1, 8]), torch.Size([5, 1]))

## Decoder Implementation in PyTorch

The init method contains the following parameters:

`num_layers` is the number of layers in LSTM

`hid_dim:` is the dimensionality of the hidden state

`emb_dim:` features of embedding vector (is the dimensionality of the embedding layer)

`dropout:` dropout random neuron to prevent overfitting

`output_dim:` Number of possible outputs (target vocabulary length)

Decoder contains the following layers:

`embedding:`

`lstm` An LSTM layer that takes the embedded input and produces hidden states of size hid_dim

`fc_out:` A linear layer that maps the LSTM output to the output dimension output_dim

`softmax:` A long softmax activation function applied to the output to obtain a probability distribution over the output values 

`dropout`

In [150]:
class Decoder(torch.nn.Module):

    def __init__(self,emb_dim,hid_dim,n_layers,output_dim,dropout_prob):

        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        

        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim,hid_dim, n_layers,dropout = dropout_prob)

        self.fc_out = nn.Linear(hid_dim,output_dim)

        self.softmax = nn.LogSoftmax(dim = 1)

        
        self.dropout = nn.Dropout(dropout_prob)



    def forward(self, input,hidden,cell):

        
        # input = [batch_size]

        # hidden_dim = [n_layers * n_directions, batch_size, hid_dim]

        # cell_dim = [n_layers * n_directions, batch_size, hid_dim]

        # n_direction in decoder will both always be 1, therefore:
        # hidden = [n layers, batch_size, hid dim]
        # context = [n layers, batch_size, hid_dim]

        if input.dim() ==1:
            input = input.unsqueeze(0)
        elif input.dim()==0:
            input = input.view(1,1)
            
        # input = [1,batch_size, emb dim] # creates the input a 3D shaped array

        embedded = self.dropout(self.embedding(input))
        print(f"embeded shape: {embedded.shape}")

        # embedded = [1,batch_size, emb_dim]

        output, (hidden,cell)  = self.lstm(embedded, (hidden,cell))

        # output = [seq_len, batch_size, hid_dim * n_direction]

        # hidden = [n_layers * n_direction,batch_size,hid_dim]

        # cell = [n_layers * n_direction,batch_size,hid_dim]

        prediction_logit = self.fc_out(output.squeeze(0))

        prediction  = self.softmax(prediction_logit)

        # prediction = [batch_size, output_dim]


        return prediction, hidden, cell
        
        

In [151]:
output_dim = 6
emb_dim = 10
hid_dim = 8
n_layers = 1
dropout = 0.5
decoder_t = Decoder(emb_dim, hid_dim,n_layers, output_dim,dropout).to(device)

In [152]:
input_t = torch.tensor([0]).to(device)

input_t.shape

torch.Size([1])

In [153]:
prediction, hidden, cell  = decoder_t(input_t,hidden_t,cell_t)

print(f"Prediction: {prediction}\nHidden: {hidden}\nCell: {cell}")

embeded shape: torch.Size([1, 1, 10])
Prediction: tensor([[-2.1600, -1.8589, -1.5200, -2.2193, -1.5989, -1.6130]],
       grad_fn=<LogSoftmaxBackward0>)
Hidden: tensor([[[-0.0759, -0.3959, -0.3013, -0.1028, -0.1011, -0.1888,  0.2365,
           0.0567]]], grad_fn=<StackBackward0>)
Cell: tensor([[[-0.4043, -0.5529, -0.7236, -0.2843, -0.1467, -0.3228,  0.3428,
           0.1406]]], grad_fn=<StackBackward0>)


Now that you have instances of both encoder and decoder, you are ready to connect them (the red box in the diagram below). First, let's see how you can pass the Hidden and Cell (the pink cell within the red box) from encoder (the green boxes container) to decoder (the orange boxes container). Looking at the diagram, you can see that the decoder also receives an input which is the previous word that it has predicted. For the first decoder cell, this input is `<bos>` token. Each decoder cell outputs a prediction and updates the cell and state to pass to the next decoder cell. prediction is a probability distribution over possible target tokens (length of target vocab).

![connection](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/ED_connection.JPG)


## Encoder-Decoder connection

In [154]:
# trg = [trg_len, batch_size]
# teacher_forcing_ratio is 0.75 you use ground-truth inputs 75% of the time
# if teacher_forcing_ratio is 0.75 you use ground truth inputs 75% of the time

teacher_forcing_ratio = 0.5
trg = torch.tensor([[0],[2],[3],[5],[1]]).to(device)

batch_size =trg.shape[1]
trg_len = trg.shape[0]

trg_vocab_size = decoder_t.output_dim


# tensor to store decoder outputs
outputs_t = torch.zeros(trg_len, batch_size, trg_vocab_size) # words_per_sentence, number_of_sentence, output_vocab_size


# send to device
hidden_t = hidden_t.to(device)
cell_t = cell_t.to(device)

# first input to the decoder is the <bos> tokens
input = trg[0,: ]


for t in range(1, trg_len):

    # loop through the trg len and generate tokens
    # decoder receives previous generated token, cell and hidden
    # decoder outputs it prediction (probability distribution for the next token) and updates hidden and cell

    output_t, hidden_t, cell_t = decoder_t(input, hidden_t, cell_t)

    # place predictions in a tensor holding predictions for each token
    outputs_t[t] = output_t

    # decide if you are going to use teacher forcing or not
    teacher_force = random.random() < teacher_forcing_ratio

    # get the highest predicted token from your predictions
    top1 = output_t.argmax(1)

    # if teacher forcing, use actual next token as next input
    # if not, use predicted token
    # input = trg[i] if teacher_force else top1
    input = trg[t] if teacher_force else top1


print(outputs_t,outputs_t.shape)

embeded shape: torch.Size([1, 1, 10])
embeded shape: torch.Size([1, 1, 10])
embeded shape: torch.Size([1, 1, 10])
embeded shape: torch.Size([1, 1, 10])
tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[-2.0548, -1.7087, -1.5305, -2.2095, -1.8622, -1.5641]],

        [[-2.0555, -1.7544, -1.5032, -2.2261, -1.8190, -1.5779]],

        [[-1.9344, -1.8208, -1.4639, -2.2149, -1.8498, -1.6305]],

        [[-1.9220, -1.6845, -1.4960, -2.1895, -2.0248, -1.6089]]],
       grad_fn=<CopySlices>) torch.Size([5, 1, 6])


In [155]:
input_  = trg[0,:]
input_,trg.shape

(tensor([0]), torch.Size([5, 1]))

In [156]:
r_chk = torch.tensor([[1,2,3,4],[5,6,7,8]])
r_chk,r_chk.shape

(tensor([[1, 2, 3, 4],
         [5, 6, 7, 8]]),
 torch.Size([2, 4]))

In [157]:
r_chk[0,:]

tensor([1, 2, 3, 4])

In [158]:
trg_vocab_size,outputs_t,outputs_t.shape


(6,
 tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
 
         [[-2.0548, -1.7087, -1.5305, -2.2095, -1.8622, -1.5641]],
 
         [[-2.0555, -1.7544, -1.5032, -2.2261, -1.8190, -1.5779]],
 
         [[-1.9344, -1.8208, -1.4639, -2.2149, -1.8498, -1.6305]],
 
         [[-1.9220, -1.6845, -1.4960, -2.1895, -2.0248, -1.6089]]],
        grad_fn=<CopySlices>),
 torch.Size([5, 1, 6]))

In [159]:
trg = torch.tensor([[0],[2],[3],[5],[1]])

In [160]:
trg,trg.shape


(tensor([[0],
         [2],
         [3],
         [5],
         [1]]),
 torch.Size([5, 1]))

In [161]:
device

device(type='cpu')

In [162]:
class Seq2Seq(nn.Module):

    def __init__(self,encoder, decoder, device, trg_vocab):

        super().__init__()

        self.encoder = encoder
        self.decoder = decoder 
        self.device = device
        self.trg_vocab = trg_vocab

        assert encoder.hid_dim == decoder.hid_dim
        "Hidden dimensions of encoder and decoder must be equal"
        assert encoder.n_layers == decoder.n_layers
        "Encoder and decoder must have equal number of layers"

    def forward(self, src, trg, teacher_forcing_ratio =0.5):

        ## src = [src len, batch_size]
        ## trg = [trg len, batch_size]
        ## teacher_forcing_ratio is 0.75, you use ground truth inputs 75% of the time

        trg_batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        ## tensor to store decoder outputs
        outputs = torch.zeros(trg_len, trg_batch_size, trg_vocab_size).to(device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        hidden = hidden.to(device)
        cell  = cell.to(device)

        # first input to the decoder is the <bos> tokens
        input = trg[0,:] # skip the first row, that represents the first token

        for t in range(1,trg_len): # Passing through each words

            # loop through the trg len and generate tokens
            # decoder receives previous generated token, cell and hidden
            # decoder outputs it prediction probability distribution for next token and updates hidden and cel

            output,hidden,cell = self.decoder(input,hidden,cell)

            # place prediction in a tensor holding predictions for each token
            outputs[t] = output

            # decide if you are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax()

            input = trg[t] if teacher_force else top1

        return outputs
            
            


        

        

In [163]:
def train(model, iterator, loss_fn, optimizer, clip):

    model.train()


    epoch_loss = 0

    # wrap iterator with tqdm for progress logging
    train_iterator = tqdm(iterator, desc = "Training",leave = False)

    for i, (src,trg) in enumerate(iterator):

        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        # trg [trg_len, batch_size]
        # output [trg_len, batch_size, output_dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1,output_dim)

        trg = trg[1:].contiguous().view(-1)

        # trg = [(trg_len-1)* batch_size]
        # output = [(trg len-1)* batch_size, output dim]

        loss = loss_fn(output, trg)

        loss.backward()


        torch.nn.utils.clip_grad_norm(model.parameters(),clip)

        optimizer.step()

        # update tqdm progress bar with the current loss
        train_iterator.set_postfix(loss = loss.item())

        epoch_loss += loss.item()


    return epoch_loss /len(list(iterator))

        

## Data Evaluation / Testing

You also need to define a function to evaluate the model. Let's go through the code and understand its components:

1. `evaluate(model, iterator, criterion)` takes three arguments:
   - `model` is the neural network model that will be evaluated.
   - `iterator` is an iterable object that provides the evaluation data in batches.
   - `criterion` is the loss function that measures the model's performance.
* Note that evaluate function do not perform any optimization on the model.

2. The function starts by setting the model to evaluation mode with `model.eval()`.

3. It initializes a variable `epoch_loss` to keep track of the accumulated loss during the evaluation.

4. The function enters a `with torch.no_grad()` block, which ensures that no gradients are computed during the evaluation. This saves memory and speeds up the evaluation process since gradients are not needed for parameter updates.

5. The function iterates over the evaluation data provided by the `iterator`. Each iteration retrieves a batch of input sequences (`src`) and target sequences (`trg`).

6. The input sequences (`src`) and target sequences (`trg`) are moved to the appropriate device (e.g., GPU) using `src = src.to(device)` and `trg = trg.to(device)`.

7. The model is then called with `output = model(src, trg, 0)` to obtain the model's predictions for the target sequences. The third argument `0` is passed to indicate that teacher forcing is turned off during evaluation.  During evaluation, teacher forcing is typically turned off to evaluate the model's ability to generate sequences based on its own predictions.

8. The `output` tensor has dimensions `[trg len, batch size, output dim]`. To calculate the loss, the tensor is reshaped to `[trg len - 1, batch size, output dim]` to remove the initial `<bos>` (beginning of sequence) token, which is not used for calculating the loss.

9. The target sequences (`trg`) are also reshaped to `[trg len - 1]` by removing the initial `<bos>` token and making it a contiguous tensor. This matches the shape of the reshaped `output` tensor.

10. The loss between the reshaped `output` and `trg` tensors is calculated using the specified `criterion`.

11. The current batch loss (`loss.item()`) is added to the `epoch_loss` variable.

12. After all the batches have been processed, the function returns the average loss per batch for the entire evaluation, calculated as `epoch_loss / len(list(iterator))`.


In [164]:
def evaluate(iterator, model ,loss_fn):

    model.eval()

    epoch_loss = 0
    test_iterator = tqdm(iterator,desc= "Testing",leave = False)


    with torch.inference_mode():
        for i, (src,trg) in enumerate(test_iterator):
    
            src, trg = src.to(device),trg.to(device)
    
            output = model(src, trg,0) # set the teacher forcing ratio as 0
    
            # trg_dim -> [trg_len,batch_size]
            # output -> [trg_len,batch_size,output_dim]
    
            # to calculate for loss calculation, we should reshape the trg and output dimention
            output_dim = output[-1]
    
            # trg = [trg_len-1*batch_size]
            # output = [trg_len-1*batch_size, output_dim]
            trg = trg[1:].contiguous().view(-1)
    
    
            output = output[1:].view(-1,output_dim)
    
            loss = loss_fn(output,trg)

            test_iterator.set_postfix(loss = loss.item())

            epoch_loss += loss.item()

        return epoch_loss/len(list(iterator))

## Data Preprocessing

Get a translation dataset called Multi30K, collate it (tokenization, numericallization, and adding BOS/EOS and padding) and create iterable batches of src and trg tensors

This leverages the predefined collate_fn to efficiently curate and ready batches for training the transformer model.

In [165]:
#!brew install wget


In [166]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Multi30K_de_en_dataloader.py'

In [167]:
from Multi30K_de_en_dataloader import get_translation_dataloaders

train_dataloader, valid_dataloader = get_translation_dataloaders(batch_size = 4)



In [168]:
src,trg = next(iter(train_dataloader))

src, trg

(tensor([[    2,     2,     2,     2],
         [    3,  5510,  5510, 12642],
         [    1,     3,     3,     8],
         [    1,     1,     1,  1701],
         [    1,     1,     1,     3]]),
 tensor([[   2,    2,    2,    2],
         [   3, 6650,  216,    6],
         [   1, 4623,  110, 3398],
         [   1,  259, 3913,  202],
         [   1,  172, 1650,  109],
         [   1, 9953, 3823,   37],
         [   1,  115,   71,    3],
         [   1,  692, 2808,    1],
         [   1, 3428, 2187,    1],
         [   1,    5,    5,    1],
         [   1,    3,    3,    1]]))

In [169]:
src.shape, trg.shape

(torch.Size([5, 4]), torch.Size([11, 4]))

In [170]:
from Multi30K_de_en_dataloader import index_to_german,index_to_eng,vocab_transform

In [171]:
data_itr = iter(train_dataloader)

# moving forward in the dataset to reach sequences of longer length for illustration purpose. 

for n in range(1000):
    german,english = next(data_itr)

    # source and target, from german to convert English


for n in range(3):
    german,english = next(data_itr)
    german = german.T

    english = english.T

    print("___________________")
    print("german")

    for g in german:
        print(index_to_german(g))

    print("____________________")
    print("english")

    for e in english:
        print(index_to_eng(e))
    

___________________
german
<bos> Personen mit schwarzen Hüten in der Innenstadt . <eos>
<bos> Eine Gruppe Menschen protestiert in einer Stadt . <eos>
<bos> Eine Gruppe teilt ihre politischen Ansichten mit . <eos>
<bos> Mehrere Personen sitzen an einem felsigen Strand . <eos>
____________________
english
<bos> People in black hats gathered together downtown . <eos> <pad> <pad> <pad>
<bos> A group of people protesting in a city . <eos> <pad> <pad>
<bos> A group is letting their political opinion be known . <eos> <pad>
<bos> A group of people are sitting on a rocky beach . <eos>
___________________
german
<bos> Zwei sitzende Personen mit Hüten und Sonnenbrillen . <eos>
<bos> Ein kleiner Junge mit Hut beim Angeln . <eos>
<bos> Diese zwei Frauen haben Spaß im Giorgio's . <eos>
<bos> Zwei kleine Kinder schlafen auf dem Sofa . <eos>
____________________
english
<bos> Two people sitting in hats and shades . <eos> <pad> <pad> <pad>
<bos> A young boy in a hat is fishing by himself . <eos>
<bos> 

## Training the Model

### initialization

In [172]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.mps.manual_seed(SEED)

### Training

enc = Encoder(INPUT_DIM ,ENC_EMB_DIM, HID_DIM , N_LAYERS, ENC_DROPOUT)
This line creates an instance of the Encoder class, which represents the encoder component of the Seq2Seq model.


dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
line creates an instance of the Decoder class, which represents the decoder component of the Seq2Seq model

model = Seq2Seq(enc,dec,device, trg_vocab = vocab_transformation['en']).to(device)
This line creates an instance of the seq2seq class

In [173]:
len(vocab_transform["de"])

19214

In [174]:
german_vocab = vocab_transform["de"]

In [175]:
german_vocab['the']

2501

In [176]:
INPUT_DIM = len(vocab_transform['de'])
OUTPUT_DIM = len(vocab_transform['en'])
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 1
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3


enc = Encoder(INPUT_DIM,ENC_EMB_DIM,HID_DIM,N_LAYERS,ENC_DROPOUT)
dec = Decoder(DEC_EMB_DIM,HID_DIM,N_LAYERS,OUTPUT_DIM,DEC_DROPOUT)

model = Seq2Seq(enc,dec,device,trg_vocab = vocab_transform['en'].to(device))

In [177]:
german_vocab = vocab_transform['de']
english_vocab = vocab_transform['en']

In [178]:
len(vocab_transform['de']),len(vocab_transform['en'])

(19214, 10837)

In [179]:
german_vocab(['meine','bruder','ist','sehr','klug'])

[7638, 0, 48, 231, 0]

In [180]:
english_vocab(['my','brother','is','very','smart'])

[2227, 2025, 10, 268, 10187]

def init_weights(m): take model as input. The purpose of this function is to initialize the weights of the nn module



In [181]:
def init_weights(m):

    for name,param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08,0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19214, 128)
    (lstm): LSTM(128, 256, dropout=0.3)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10837, 128)
    (lstm): LSTM(128, 256, dropout=0.3)
    (fc_out): Linear(in_features=256, out_features=10837, bias=True)
    (softmax): LogSoftmax(dim=1)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (trg_vocab): Vocab()
)

In [182]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



print(f"The odel has {count_parameters(model):} trainable parameters")

The odel has 7422165 trainable parameters


In [183]:
optimizer = torch.optim.Adam(model.parameters())

PAD_IDX = vocab_transform['en'].get_stoi()['<pad>']

loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [184]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time-start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins,elapsed_secs

In [185]:
torch.mps.empty_cache()

N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')
best_train_loss = float('inf')

train_losses = []
valid_losses = []

train_PPLs = []
valid_PPLs = []

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model,train_dataloader, loss_fn,optimizer,CLIP)
    train_ppl = math.exp(train_loss)

    valid_loss =evaluate(valid_dataloader,model,loss_fn)

    valid_ppl = math.exp(valid_loss)

    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time,end_time)


    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'RNN-TR-model.pth')



    train_losses.append(train_loss)
    train_PPLs.append(train_ppl)
    valid_losses.append(valid_loss)
    valid_PPLs.append(valid_ppl)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {train_ppl:7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {valid_ppl:7.3f}')


Training:   0%|          | 0/7250 [00:00<?, ?it/s]

embeded shape: torch.Size([1, 4, 128])
embeded shape: torch.Size([1, 1, 128])


RuntimeError: Expected hidden[0] size (1, 1, 256), got [1, 4, 256]

In [ ]:
!install tqdm
from tqdm.auto import tqdm